In [1]:
import qdrant_client
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.indices import MultiModalVectorStoreIndex
import yaml


In [3]:
def get_document_metadata(filepath):
    with open(filepath, 'r') as file:
        content = file.read()
        _, front_matter, _ = content.split('---', 2)
        data = yaml.safe_load(front_matter)
    return data

In [4]:
# Create a local Qdrant vector store
client = qdrant_client.QdrantClient(path="qdrant_db")

In [5]:
text_store = QdrantVectorStore(
    client=client, collection_name="text_collection"
)
image_store = QdrantVectorStore(
    client=client, collection_name="image_collection"
)
storage_context = StorageContext.from_defaults(
    vector_store=text_store, image_store=image_store
)

In [ ]:
client.delete_collection(collection_name="text_collection")
client.delete_collection(collection_name="image_collection")

In [8]:
documents = SimpleDirectoryReader(input_dir="docs", file_metadata=get_document_metadata).load_data()

In [ ]:
client.close()

In [9]:
# Create the MultiModal index
index = MultiModalVectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

In [10]:
test_query = "How much did prices of retail spaces increase by in 4th Quarter 2023?"
# generate  retrieval results
retriever = index.as_retriever(similarity_top_k=3)
retrieval_results = retriever.retrieve(test_query)
from llama_index.core.response.notebook_utils import display_source_node
for res_node in retrieval_results:
    display_source_node(res_node, source_length=1000, show_source_metadata=True)

**Node ID:** b1b61e4c-3a54-4483-b47d-e7500f559ac6<br>**Similarity:** 0.9081193440179913<br>**Text:** Prices and Rentals

Prices of retail space increased by 1.2% in 4th Quarter 2023, compared with the 0.6% increase in the previous quarter. Rentals of retail space decreased by 0.1% in 4th Quarter 2023, compared with the 0.5% increase in the previous quarter. For the whole of 2023, prices of retail space increased by 1.2%, compared with the decrease of 7.8% in 2022, while rentals of retail space increased by 0.4%, compared with the decrease of 2.4% in 2022.<br>**Metadata:** {'title': 'Release of 4th Quarter 2023 Real Estate Statistics', 'date': '2024-01-26', 'source_link': 'https://www.ura.gov.sg/Corporate/Media-Room/Media-Releases/pr24-03'}<br>